In [82]:
#Importing required liberaries
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import requests
from selenium.webdriver.support import expected_conditions as EC

In [83]:
#Declaring link of website 
link="https://www.makaan.com/"
requests.get(link)

<Response [200]>

In [84]:
s = Service(r"C:\Users\bhupe\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe") # location of Web driver
driver = webdriver.Chrome(service=s) #Intialising Web driver
driver.maximize_window() 
driver.get(link)
city="Bangalore"  ##-----Put name of city for which you want to scrap data( check spellings) 


In [78]:
# Finding *Serach Bar* Path in website and Pushing city as "Bangalore"
search_bar_xpath = "/html/body/div[1]/main/div/section[1]/div[1]/div/div[3]/div[1]/div/div/div/div[2]/input"
search_bar = driver.find_element(By.XPATH, search_bar_xpath)
search_bar.send_keys(city) 


# Wait for the "result-row" element to be clickable and then click it
result_row = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,"result-row")))
result_row.click()



# Finding  *Search Button* elment and then click on it
search_button_xpath = "/html/body/div[1]/main/div/section[1]/div[1]/div/div[3]/div[3]/span[2]"
search_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, search_button_xpath)))
search_button.click()


driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll to bottom of page



# Wait for the "page numbers" element to be visible and then extract the number of pages
page_numbers_xpath = "/html/body/div[1]/main/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div/ul/li[7]/a"
page_numbers_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, page_numbers_xpath)))
total_page = int(page_numbers_element.text)
print(f'No of total pages available on wesite for {city}:-->{total_page}')


No of total pages available on wesite for Bangalore:-->2163


In [85]:
pages=int(input("Enter the number of pages you want to scrap:"))  # Numnber of pages should not exceed the number of pages on website.

In [86]:

   
dic={"property":[],"location":[],"price":[],"area":[],"furnished status":[],"additional details":[]}
print("Number of pages Scrapped")
for page in range(1,pages+1):
    if page % 50 == 0:
        time.sleep(30)

    driver.get(f"https://www.makaan.com/{city}-residential-property/buy-property-in-{city}-city?page={page}")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    listings = driver.find_elements(By.CLASS_NAME, "cardholder")  

    for listing in listings:
        try:
            location = listing.find_element(By.CLASS_NAME, "loclink").text
        except:
            location = "NULL"
        
        try:
            property_name = listing.find_element(By.CLASS_NAME, "title-line").text
        except:
            property_name = "NULL"
        
        try:
            price = listing.find_element(By.CLASS_NAME, "price").text
        except:
            price = "NULL"
        
        try:
            area = listing.find_element(By.CLASS_NAME, "size").text
        except:
            area = "NULL"
        
        try:
            furnished_status = listing.find_element(By.CLASS_NAME, "hcol.w44").text
        except:
            furnished_status = "NULL"
        
        try:
            additional_details = listing.find_element(By.CLASS_NAME, "listing-details").text
        except:
            additional_details = "NULL"

        # Append all values to the dictionary
        dic["location"].append(location)
        dic["property"].append(property_name)
        dic["price"].append(price)
        dic["area"].append(area)
        dic["furnished status"].append(furnished_status)
        dic["additional details"].append(additional_details)

       


        # Save to CSV when the data reaches 5000 entries
        if len(dic["location"]) == 5000:
            i=1
            file = pd.DataFrame(dic)
            print(f'-----------FILE ::: {i}--{5000*i} <-- Data Extracted')
            print(file.head())
            file.to_csv(f"makan{i}.csv", index=False)
            dic = {"property": [], "location": [], "price": [], "area": [], "furnished status": [], "additional details": []}

    #printing number of pages to get location of driver
    print(page,end=" ")


Number of pages Scapped:  
1 2 3 4 5 

In [88]:
df=pd.DataFrame(dic)
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   property            97 non-null     object
 1   location            97 non-null     object
 2   price               97 non-null     object
 3   area                97 non-null     object
 4   furnished status    97 non-null     object
 5   additional details  97 non-null     object
dtypes: object(6)
memory usage: 4.7+ KB
None


,property,location,price,area,furnished status,additional details
0,2 BHK Apartment in Mantri Serenity,"Subramanyapura, Bangalore",94.93 L,1040,Ready to move Construction Status,2 - 3 years oldNew
1,2 BHK Apartment in Elegant Tranquility,"Rajarajeshwari Nagar, Bangalore",95.21 L,1045,Under Construction Construction Status,Possession by Jul 2025New
2,Residential Plot in Vinyasa Greenland,"Bidadi, Bangalore",28.63 L,1146,New New/Resale,0 - 1 year old
3,2 BHK Apartment in Ma Sarada Upavan,"Bommasandra, Bangalore",63.93 L,870,Under Construction Construction Status,Possession by Aug 2024New
4,2 BHK Apartment in Srivari Forest Breeze,"Uttarahalli, Bangalore",81.18 L,1160,Under Construction Construction Status,Possession by Sep 2024New
...,...,...,...,...,...,...
92,5 BHK Independent House,"RR Nagar, Bangalore",2.95 Cr,3500,Under Construction Construction Status,5 BathroomsResale
93,Residential Plot,"Bannerughatta, Bangalore",45 L,1200,Resale New/Resale,7 - 8 years old3 open sidesNorthEast facing
94,3 BHK Apartment,"Hoodi, Bangalore",1.75 Cr,2030,Under Construction Construction Status,3 BathroomsResale
95,2 BHK Apartment in Prestige Misty Waters,"Hebbal, Bangalore",1.5 Cr,1350,Ready to move Construction Status,6 - 7 years old2 BathroomsResale
